# Scratch work for TensorFlow skills test (image recognition)

- Author: Chris Hodapp
- Date: 2017-11-28
- For SharpestMinds/Yazabi

## Links:

- [The Street View House Numbers (SVHN) Dataset](http://ufldl.stanford.edu/housenumbers)
  - Using [train_32x32.mat](http://ufldl.stanford.edu/housenumbers/train_32x32.mat) and [test_32x32.mat](http://ufldl.stanford.edu/housenumbers/test_32x32.mat) in `./data` directory